In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from minigpt.dataset import ShakespeareDataset

In [3]:
dataset = ShakespeareDataset("data/shakespeare.txt")

In [4]:
print(dataset.text()[0:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(dataset.text())))
vocab_size = len(chars)

In [6]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [7]:
data = torch.tensor(encode(dataset.text()), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

In [10]:
x, y = get_batch('train')
print(x)
print(y)

tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])


In [46]:
device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'
elif torch.backends.mps.is_available():
  device = 'mps'
print(f"Using device: {device}")

Using device: mps


In [ ]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding(idx) # B, T, C
    B, T, C = logits.shape

    if targets is None:
      loss = None
    else:
      targets = targets.view(B*T)
      logits = logits.view(B*T, C)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)
      logits = logits[:, -1, :] # B, C
      probs = F.softmax(logits, dim=-1) # B, C
      idx_next = torch.multinomial(probs, num_samples=1) # B, 1
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

In [40]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(x, targets=y)
print(logits.shape)
print(loss.item())

torch.Size([32, 65])
4.638251304626465


In [47]:
input_data = torch.zeros((1, 1), dtype=torch.long).to(device)
print(decode(m.generate(input_data, max_new_tokens=100)[0].tolist()))


-r y te k coput bainuthas al chie athotorde us m wh.
QUS:
Bukerene my n wanofeCLur nd blkerd FReps t


In [42]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [48]:
@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(100)
    for k in range(100):
      x, y = get_batch(split)
      logits, loss = m(x, targets=y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out

In [ ]:
m.to(device)
batch_size = 32
for step in range(10000):
  xb, yb = get_batch('train')
  xb, yb = xb.to(device), yb.to(device)
  if step % 3000 == 0:
    losses = estimate_loss()
    print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  logits, loss = m(xb, targets=yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if step % 1000 == 0:
    print(f"step {step}: loss {loss.item()}")

step 0: loss 4.655232906341553
step 1000: loss 3.629887580871582
step 2000: loss 3.0939760208129883
step 3000: loss 2.786675453186035
step 4000: loss 2.703126907348633
step 5000: loss 2.566441535949707
step 6000: loss 2.4160685539245605
step 7000: loss 2.477687358856201
step 8000: loss 2.5292487144470215
step 9000: loss 2.3415887355804443


In [53]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B, T, C)
print(x.shape)

torch.Size([4, 8, 2])


In [54]:
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xbow[b, t] = x[b, :t+1].mean(0)

In [67]:
a = torch.tril(torch.ones((3,3)))
a /= torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[1., 9.],
        [0., 1.],
        [9., 8.]])
tensor([[1.0000, 9.0000],
        [0.5000, 5.0000],
        [3.3333, 6.0000]])


In [71]:
# version using matrix multiplication
wei = torch.tril(torch.ones((T,T)))
wei /= wei.sum(1, keepdim=True)

xbow2 = wei @ x # (T, T) @ (B, T, C) -> (B, T, C)
print(torch.allclose(xbow, xbow2))

True


In [73]:
# using softmax
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
print(torch.allclose(xbow, xbow3))

True


# Self attention

In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B, T, C)
print(x.shape)

head_size = 16
key = nn.Linear(C, head_size, bias=False) # B, C, 16
query = nn.Linear(C, head_size, bias=False) # B, C, 16
value = nn.Linear(C, head_size, bias=False) # B, C, 16

k = key(x)   # B, T, 16
q = query(x) # B, T, 16
v = value(x) # B, T, 16

wei = q @ k.transpose(-2, -1) * (head_size ** -0.5) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones((T,T)))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v

print(wei[0])

torch.Size([4, 8, 32])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)
